# Hill Cipher

**Ricardo Andrés Calvo Méndez**

Introducción a la criptografía y a la seguridad de la información

## Algorithm description

The idea is to take $t$ linear combinations of the $t$ alphabetic characters in one plaintext element, thus producing the $m$ alphabetic characters in one cipher element.

## Implementation in Python

### Dependences

In [1]:
# Using 'numpy' to easy-way matrix representacion
import numpy as np

### Utilities

In [2]:
# Extended Euclidean Algorithm
def EEA(a, b):
  if b == 0:
    return a, 1, 0
  q = a // b
  dp, xp, yp = EEA(b, a % b)
  return dp, yp, xp - q * yp

In [3]:
# Greatest common divisor
def GCD(a, b):
  d, _, _ = EEA(a, b)
  return d

In [4]:
# Modular inverse of a given matrix
def modular_inverse(matrix, module):
  determinant = np.linalg.det(matrix).round()
  cofactor_matrix = np.linalg.inv(matrix).T * determinant
  adjoint = np.transpose(cofactor_matrix)
  _ , modular_determinant, _ = EEA(determinant, module)
  modular_inverse = (modular_determinant * adjoint) % module
  return np.matrix.round(modular_inverse).astype(int)

### Key generator

In [5]:
# Function to generate encrypt and decrypt for this algorithm. This key are a modular matrix and its inverse
def generate_key(size):
  while True:
    encrypt_key = np.random.randint(26, size=(size, size))
    determinant = np.linalg.det(encrypt_key).round()
    if determinant != 0 and GCD(determinant, 26) == 1:
      break
  decrypt_key = modular_inverse(encrypt_key, 26)
  return encrypt_key, decrypt_key

### Encrypy function

In [6]:
# Function to encrypt a given message using the Hill cipher
def encrypt(message, key):
  alphabet = 'ABCDEFGHIJKLMNOPQRSTUVWXYZ'
  message = message.upper()
  message = message.replace(' ', '')
  ciphertext = ''
  size = key.shape[0]
  message += 'X' * ((size - len(message)) % size)
  for index in range(0, len(message), size):
    m = np.array([alphabet.index(x) for x in message[index: index + size]])
    c = m @ key
    ciphertext += ''.join([alphabet[x % 26] for x in c])
  return ciphertext

### Decrypt function

In [7]:
# Function to decrypt a given covered message using the Hill cipher
def decrypt(ciphertext, key):
  alphabet = 'ABCDEFGHIJKLMNOPQRSTUVWXYZ'
  ciphertext = ciphertext.upper()
  ciphertext = ciphertext.replace(' ', '')
  message = ''
  size = key.shape[0]
  for index in range(0, len(ciphertext), size):
    c = np.array([alphabet.index(x) for x in ciphertext[index: index + size]])
    m = c @ key
    message += ''.join([alphabet[x % 26] for x in m])
  return message

### Hill function


In [8]:
# Hill Cipher 
def hill(message, key, decrypt_mode=True):
  if decrypt_mode:
    return decrypt(message, key)
  else:
    return encrypt(message, key)

## Example

In [9]:
# Define 't' as the size of the square matrix used as key in this algorithm 
t = 2
print("Size of key (t): \n", t)

# Generate a random matrix and its modular inverse, used as key in this algorithm
encrypt_key, decrypt_key = generate_key(t)
print("Encrypt key : \n", encrypt_key)
print("Decrypt key : \n", decrypt_key)

# Define the message to encrypt using this algorithm
message = 'I WILL ENCRYPT THIS MESSAGE TO COVER IMPORTANT INFORMATION' 
print("Message: \n", message)

# Cipher the message using the Hill algorithm
ciphertext = hill(message, encrypt_key, decrypt_mode=False)
print("Cipher text: \n", ciphertext)

# Recover the original message using the Hill algorithm (undebugged message)
recovered_message = hill(ciphertext, decrypt_key, decrypt_mode=True)
print("Recovered message (undebugged): \n", recovered_message)

Size of key (t): 
 2
Encrypt key : 
 [[16  1]
 [23  4]]
Decrypt key : 
 [[ 2 19]
 [21  8]]
Message: 
 I WILL ENCRYPT THIS MESSAGE TO COVER IMPORTANT INFORMATION
Cipher text: 
 KSRAIBUVSJBNXVWCYCAMIYHCKWFUNUOEQTHPNAUZLHREKMUZDO
Recovered message (undebugged): 
 IWILLENCRYPTTHISMESSAGETOCOVERIMPORTANTINFORMATION
